In [1]:
%cd ../..

/Users/davideleo/Desktop/Projects/research/papers/fl_wavelet_v0


# Malicious clients analysis
## FedAvg w/ malicious clients 

In [2]:
import random
import torch 
import numpy as np 
import matplotlib.pyplot as plt
from src.data.fashion_mnist import get_federation 
from src.data.attacks import GaussianBlur, GaussianNoise
from src.federated_learning.detection.fldetector import Client, Server 
from src.models.neural_networks import LeNet5
from src.models.metrics import Accuracy, WeightedAccuracy

random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)

# Federation
federation = get_federation(
    num_shards = 100,
    alpha = 1000,
    attacks = [GaussianNoise(sigma = .5), GaussianBlur(kernel_size = 11)],
    attacks_proba = 0.4
)

clients = [
    Client(
        train_dataset = dataset["train"],
        test_dataset = dataset["test"],
        distribution = dataset["distribution"],
        batch_size = 64,
        device = "cpu"
    ) for dataset in federation
]

benign_clients = [client for dataset, client in zip(federation, clients) if len(dataset["id"].split(".")) == 1] 

## FedAvg w/o malicious clients

In [3]:
# Default experiments: new_seed = None -> [None, 7, 365]
new_seed = 365

if new_seed is not None: 
    random.seed(new_seed)
    np.random.seed(new_seed)
    torch.random.manual_seed(new_seed)

# Trainin
server = Server(
    clients = clients,
    participation_rate = 0.1, 
    model = LeNet5(),
    window_size = 10,
    batch_size = 10
)

benign_train_results = server.train(
    num_rounds = 500,
    num_local_epochs = 1,
    criterion = torch.nn.CrossEntropyLoss(),
    optimizer_class = torch.optim.Adam, 
    optimizer_params = {"lr": 1e-3},
    evaluation_step = 501,
    metrics = dict()
)

# Evaluation 
evaluation_results = server.evaluate(
    criterion = torch.nn.CrossEntropyLoss(),
    metrics = {"acc": Accuracy(), "wacc": WeightedAccuracy()}
)

print(evaluation_results["server"])

100%|██████████| 500/500 [12:09<00:00,  1.46s/it]


{'loss': 0.5906098579863707, 'metrics': {'acc': 0.7881666690508524, 'wacc': 0.7881666710575421}}


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Setup clients
domains_wst = []
labels = []

for d in federation: 
    if "." in d["id"]: 
        labels.append(1)
    else:
        labels.append(0)

with torch.no_grad(): 
    pred = np.zeros(100)
    if len(server.malicious_clients) > 0:
        pred[np.array(server.malicious_clients)] = 1
    print("Accuracy: ", accuracy_score(labels, pred))
    print("Precision: ", precision_score(labels, pred, pos_label = 1))
    print("Recall:  ", recall_score(labels, pred, pos_label = 1))
    print("F1-Score: ", f1_score(labels, pred, average = "binary"))
    print(torch.arange(100)[pred == 1])

Accuracy:  0.6
Precision:  0.0
Recall:   0.0
F1-Score:  0.0
tensor([], dtype=torch.int64)
